# Notes and Summary of Kernel 

This is a experiment on feature extraction and feature selection, the inital idea and part of the code came from this fantastic notebook from [JEROENVDD](https://www.kaggle.com/code/jeroenvdd/tsflex-x-tsfresh-feature-extraction) and the [tsflex github](https://github.com/predict-idlab/tsflex)

I then realised that these features were already provided by Broccoli beef, who has a [fantastic notebook ](https://www.kaggle.com/code/siukeitin/tps042022-fe-2500-features-with-tsfresh-catch22) and the dataset available as output

## To do 
Calibration - yes/ no

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import seaborn as sns 

#from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost  import XGBClassifier

from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.model_selection import train_test_split, GroupKFold, KFold
from sklearn.metrics import roc_auc_score, auc

import gc


In [ ]:
EPOCHS = 4096
FOLDS = 10

DROP_SENSOR= True

SCALING = True

CALIBRATION = True

Additional features from [broccoli beef ](kaggle.com/code/siukeitin/tps042022-fe-2500-features-with-tsfresh-catch22/notebook)

In [ ]:
train_original = pd.read_csv("../input/tabular-playground-series-apr-2022/train.csv")
test_original = pd.read_csv("../input/tabular-playground-series-apr-2022/test.csv")

train_feats = pd.read_csv("../input/tps042022-fe-2500-features-with-tsfresh-catch22/tps042022_train_r.csv")
test_feats = pd.read_csv("../input/tps042022-fe-2500-features-with-tsfresh-catch22/tps042022_test_r.csv")
train_lables = pd.read_csv("../input/tabular-playground-series-apr-2022/train_labels.csv")
sub= pd.read_csv("../input/tabular-playground-series-apr-2022/sample_submission.csv", index_col = 0)

In [ ]:
sensor_cols = [col for col in train_original.columns if "sensor" in col]
sensor_cols

In [ ]:
print("train shape:",train_feats.shape)
print("test shape:",test_feats.shape)
train_feats

# Added features 


In [ ]:
train_pivoted = train_original.pivot(index=['sequence', 'subject'], columns='step', values=sensor_cols)
test_pivoted = test_original.pivot(index=['sequence', 'subject'], columns='step', values=sensor_cols)

Sensor_02 features from [AMBROSM](https://www.kaggle.com/code/ambrosm/tpsapr22-best-model-without-nn#Cross-validation  ) which significantly improve the score

In [ ]:
import scipy

def add_features(df):
    #new_df = pd.DataFrame()
    #df_pivot = df.pivot(index = ["sequence","subject"], columns ="step", values = sensor_cols)
    
    for col in sensor_cols:
        df[f"mean_{col}"] = df[col].mean(axis = 1)
#         df[f"median_{col}"] = df[col].median(axis = 1)
        df[f"std_{col}"] = df[col].std(axis = 1)
#         df[f"variance_{col}"] = df[col].std(axis = 1)
#         df[f"max_{col}"] = df[col].max(axis = 1)
#         df[f"min_{col}"] = df[col].min(axis = 1)
        df[f"max-min_{col}"] = df[col].max(axis = 1) - df[col].min(axis = 1)
        df[f"q01_{col}"] = df[col].quantile(q= 0.01, axis =1)
        df[f"q25_{col}"] = df[col].quantile(q= 0.25, axis =1) 
        df[f"q50_{col}"] = df[col].quantile(q= 0.5, axis =1)
        df[f"q75_{col}"] = df[col].quantile(q= 0.75, axis =1)
        df[f"q95_{col}"] = df[col].quantile(q= 0.95, axis =1)
        df[f"q99_{col}"] = df[col].quantile(q= 0.99, axis =1)
        df[f"skew_{col}"] = df[col].skew( axis =1)
        
                  
        df[col + '_iqr'] = scipy.stats.iqr(df[col], axis=1)
        df[col + '_sm'] = np.nan_to_num(df[f"std_{col}"] / 
                                               df[f"mean_{col}"].abs()).clip(-1e30, 1e30)
        df[f"kurtosis_{col}"] = scipy.stats.kurtosis(df[col], axis=1)
    
    df['sensor_02_up'] = (df.sensor_02.diff(axis=1) > 0).sum(axis=1)
    df['sensor_02_down'] = (df.sensor_02.diff(axis=1) < 0).sum(axis=1)
    df['sensor_02_upsum'] = df.sensor_02.diff(axis=1).clip(0, None).sum(axis=1)
    df['sensor_02_downsum'] = df.sensor_02.diff(axis=1) .clip(None, 0).sum(axis=1)
    df['sensor_02_upmax'] = df.sensor_02.diff(axis=1).max(axis=1)
    df['sensor_02_downmax'] = df.sensor_02.diff(axis=1).min(axis=1)
    df['sensor_02_upmean'] = np.nan_to_num(df['sensor_02_upsum'] / df['sensor_02_up'], posinf=40)
    df['sensor_02_downmean'] = np.nan_to_num(df['sensor_02_downsum'] / df['sensor_02_down'], neginf=-40)
    
    
    ## Trying as per my EDA
    df['sensor_02_mean_up'] = df["sensor_02"].mean(axis = 1) > -0.2
    df['sensor_02_mean_up'] = df['sensor_02_mean_up'].astype(int)
    
    df['sensor_02_std_up'] = df["sensor_02"].std(axis = 1) > 2.0
    df['sensor_02_std_up']  = df['sensor_02_std_up'] .astype(int)

    df['sensor_12_std_up'] = df["sensor_12"].std(axis = 1) > 39
    df['sensor_12_std_up'] =  df['sensor_12_std_up'].astype(int)
    
    df['sensor_04_std_up'] = df["sensor_04"].std(axis = 1) > 1.68
    df['sensor_04_std_up'] = df['sensor_04_std_up'].astype(int)
    
    
    #df = df.merge(new_df, how = "left", on = "sequence")   
    return df

add_features(train_pivoted)
add_features(test_pivoted)

In [ ]:
print("train pvt shape:",train_pivoted.shape)
print("test pvt  shape:",test_pivoted.shape)

In [ ]:
if DROP_SENSOR:
    print("dropping raw sensor data")
    train_pivoted = train_pivoted.drop(sensor_cols,axis =1)
    test_pivoted = test_pivoted.drop(sensor_cols,axis =1)

In [ ]:
print("train pvt shape:",train_pivoted.shape)
print("test pvt  shape:",test_pivoted.shape)

In [ ]:
train_feats = train_feats.merge(train_pivoted,right_on = ["sequence", "subject"] , left_on = [train_pivoted.index.get_level_values(0), train_pivoted.index.get_level_values(1)])
test_feats = test_feats.merge(test_pivoted,right_on = ["sequence", "subject"] , left_on = [test_pivoted.index.get_level_values(0), test_pivoted.index.get_level_values(1)])

In [ ]:
print("train shape:",train_feats.shape)
print("test shape:",test_feats.shape)
train_feats

## Drop Cols with 0 imporatance
from previous runs 

In [ ]:
drop_cols = ['sensor_06__permutation_entropy__dimension_7__tau_1', 'sensor_10__c22_CO_f1ecac', 'sensor_07__fourier_entropy__bins_3', 'sensor_09__c22_CO_f1ecac',
             'sensor_01__partial_autocorrelation__lag_3', 'sensor_01__number_cwt_peaks__n_1', 'sensor_09__c22_CO_FirstMin_ac', 'sensor_07__fourier_entropy__bins_2',
             'sensor_10__c22_CO_FirstMin_ac', 'sensor_07__benford_correlation', 'sensor_06__permutation_entropy__dimension_5__tau_1', 'sensor_06__fourier_entropy__bins_3', 
             'sensor_07__agg_autocorrelation__f_agg_"var"__maxlag_10', 'sensor_06__fourier_entropy__bins_5', 'sensor_01__fourier_entropy__bins_3', 
             'sensor_06__number_cwt_peaks__n_1', 'sensor_09__c22_SB_BinaryStats_diff_longstretch0', 'sensor_05__variation_coefficient', 'sensor_01__fourier_entropy__bins_5', 
             'sensor_02__c22_PD_PeriodicityWang_th0_01', 'sensor_01__partial_autocorrelation__lag_8', 'sensor_12__c22_SB_BinaryStats_mean_longstretch1', 
             'sensor_02__fourier_entropy__bins_2', 'sensor_11__c22_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1', 'sensor_11__c22_SP_Summaries_welch_rect_centroid', 
             'sensor_12__c22_CO_f1ecac', 'sensor_12__c22_CO_HistogramAMI_even_2_5', 'sensor_04__augmented_dickey_fuller__attr_"teststat"__autolag_"AIC"', 
             'sensor_02__quantile__q_0.5', 'sensor_01__permutation_entropy__dimension_3__tau_1', 'sensor_03__autocorrelation__lag_2', 'sensor_03__permutation_entropy__dimension_7__tau_1', 
             'sensor_03__permutation_entropy__dimension_6__tau_1', 'sensor_03__number_cwt_peaks__n_1', 'sensor_03__fourier_entropy__bins_2', 'sensor_03__fourier_entropy__bins_3', 
             'sensor_04__fourier_entropy__bins_2', 'sensor_02__autocorrelation__lag_3', 'sensor_11__c22_IN_AutoMutualInfoStats_40_gaussian_fmmi', 'sensor_11__c22_CO_Embed2_Dist_tau_d_expfit_meandiff', 
             'sensor_02__augmented_dickey_fuller__attr_"teststat"__autolag_"AIC"', 'sensor_11__c22_SB_BinaryStats_mean_longstretch1', 'sensor_11__c22_MD_hrv_classic_pnn40', 
             'sensor_05__augmented_dickey_fuller__attr_"teststat"__autolag_"AIC"', 'sensor_11__c22_CO_HistogramAMI_even_2_5', 'sensor_11__c22_CO_FirstMin_ac', 'sensor_11__c22_CO_f1ecac', 
             'sensor_01__permutation_entropy__dimension_7__tau_1', 'sensor_01__agg_autocorrelation__f_agg_"ptp"__maxlag_10', 'sensor_01__permutation_entropy__dimension_6__tau_1', 
             'sensor_05__fourier_entropy__bins_3', 'sensor_05__partial_autocorrelation__lag_4', 'sensor_01__permutation_entropy__dimension_5__tau_1', 'sensor_07__fourier_entropy__bins_5', 
             'sensor_08__c22_CO_FirstMin_ac', 'sensor_08__c22_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1', 'sensor_11__partial_autocorrelation__lag_3', 'sensor_12__fourier_entropy__bins_2', 
             'sensor_12__augmented_dickey_fuller__attr_"teststat"__autolag_"AIC"', 'sensor_11__variation_coefficient', 'sensor_04__c22_IN_AutoMutualInfoStats_40_gaussian_fmmi', 
             'sensor_04__c22_FC_LocalSimple_mean1_tauresrat', 'sensor_11__permutation_entropy__dimension_6__tau_1', 'sensor_11__number_cwt_peaks__n_5', 'sensor_00__permutation_entropy__dimension_7__tau_1', 'sensor_11__fourier_entropy__bins_3', 'sensor_05__c22_CO_f1ecac', 'sensor_05__c22_CO_FirstMin_ac', 'sensor_02__ar_coefficient__coeff_0__k_10', 'sensor_02__ar_coefficient__coeff_9__k_10', 'sensor_02__ar_coefficient__coeff_10__k_10', 'sensor_02__cwt_coefficients__coeff_54__w_2__widths_(2, 5, 10, 20)', 'sensor_12__number_cwt_peaks__n_5', 'sensor_00__c22_CO_f1ecac', 'sensor_04__c22_CO_FirstMin_ac', 'sensor_04__c22_CO_f1ecac', 'sensor_00__c22_CO_FirstMin_ac', 'sensor_00__c22_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1', 'sensor_01__c22_CO_f1ecac', 'sensor_01__c22_CO_FirstMin_ac', 'sensor_03__c22_SB_BinaryStats_diff_longstretch0', 'sensor_01__c22_SB_BinaryStats_mean_longstretch1', 'sensor_03__c22_IN_AutoMutualInfoStats_40_gaussian_fmmi', 'sensor_01__c22_IN_AutoMutualInfoStats_40_gaussian_fmmi', 'sensor_03__c22_SB_TransitionMatrix_3ac_sumdiagcov', 'sensor_01__c22_FC_LocalSimple_mean1_tauresrat', 'sensor_01__c22_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1', 'sensor_03__c22_CO_FirstMin_ac', 'sensor_05__c22_FC_LocalSimple_mean1_tauresrat', 'sensor_08__ar_coefficient__coeff_2__k_10', 'sensor_05__c22_SB_BinaryStats_diff_longstretch0', 'sensor_07__c22_CO_FirstMin_ac', 'sensor_00__permutation_entropy__dimension_6__tau_1', 'sensor_08__c22_FC_LocalSimple_mean1_tauresrat', 'sensor_08__c22_SB_BinaryStats_mean_longstretch1', 'sensor_08__c22_MD_hrv_classic_pnn40', 'sensor_07__permutation_entropy__dimension_6__tau_1', 'sensor_03__c22_CO_f1ecac', 'sensor_08__c22_CO_f1ecac', 'sensor_07__permutation_entropy__dimension_7__tau_1', 'sensor_00__partial_autocorrelation__lag_5', 'sensor_08__augmented_dickey_fuller__attr_"usedlag"__autolag_"AIC"', 'sensor_07__c22_FC_LocalSimple_mean1_tauresrat', 'sensor_08__fourier_entropy__bins_2', 'sensor_08__index_mass_quantile__q_0.2', 'sensor_08__number_crossing_m__m_0', 'sensor_07__c22_CO_f1ecac', 'sensor_10__augmented_dickey_fuller__attr_"teststat"__autolag_"AIC"', 'sensor_08__number_cwt_peaks__n_5', 'sensor_06__c22_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1', 'sensor_06__c22_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1', 'sensor_08__permutation_entropy__dimension_7__tau_1', 'sensor_09__benford_correlation', 'sensor_06__c22_FC_LocalSimple_mean1_tauresrat', 'sensor_06__c22_IN_AutoMutualInfoStats_40_gaussian_fmmi', 'sensor_09__cid_ce__normalize_True', 'sensor_09__number_crossing_m__m_0', 'sensor_09__permutation_entropy__dimension_7__tau_1', 'sensor_06__c22_CO_FirstMin_ac', 'sensor_06__c22_CO_f1ecac', 'sensor_05__c22_SP_Summaries_welch_rect_centroid', 'sensor_09__variation_coefficient', 'sensor_03__fourier_entropy__bins_5'
             ,('sensor_12_std_up', ''), ('sensor_02_mean_up', ''), 'sensor_04__number_crossing_m__m_0', ('sensor_02_std_up', ''), ('sensor_09_sm', ''),
             'sensor_05__autocorrelation__lag_6', 'sensor_11__fourier_entropy__bins_2', ('sensor_11_sm', ''), 'sensor_07__number_cwt_peaks__n_1', 
             'sensor_09__fourier_entropy__bins_2', 'sensor_05__permutation_entropy__dimension_7__tau_1', 'sensor_06__autocorrelation__lag_5', 
             'sensor_05__permutation_entropy__dimension_6__tau_1', ('q75_sensor_12', ''), 'sensor_06__permutation_entropy__dimension_6__tau_1', 
             'sensor_10__fourier_entropy__bins_2', 'sensor_05__permutation_entropy__dimension_5__tau_1', 'sensor_09__partial_autocorrelation__lag_9', 
             ('std_sensor_11', ''), 'sensor_09__index_mass_quantile__q_0.1', 'sensor_05__partial_autocorrelation__lag_1', 'sensor_05__number_cwt_peaks__n_5', 
             'sensor_10__cid_ce__normalize_False', ('sensor_12_sm', ''), 'sensor_09__fourier_entropy__bins_3', 'sensor_10__partial_autocorrelation__lag_5', 
             ('sensor_00_sm', ''), 'sensor_11__permutation_entropy__dimension_7__tau_1', ('mean_sensor_03', ''), 'sensor_06__c22_SB_BinaryStats_diff_longstretch0', 
             'sensor_06__c22_SP_Summaries_welch_rect_centroid', 'sensor_06__c22_FC_LocalSimple_mean3_stderr', 'sensor_07__c22_SB_TransitionMatrix_3ac_sumdiagcov',
             ('max-min_sensor_04', ''), ('std_sensor_04', ''), 'sensor_00__fourier_entropy__bins_2', 'sensor_07__c22_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1', 
             ('sensor_03_sm', ''), 'sensor_09__c22_PD_PeriodicityWang_th0_01', 'sensor_09__c22_IN_AutoMutualInfoStats_40_gaussian_fmmi', 'sensor_09__c22_FC_LocalSimple_mean3_stderr', 
             'sensor_05__c22_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1', 'sensor_10__c22_MD_hrv_classic_pnn40', 'sensor_00__agg_autocorrelation__f_agg_"var"__maxlag_10', 
             'sensor_00__agg_autocorrelation__f_agg_"ptp"__maxlag_10', ('q75_sensor_02', ''), 'sensor_10__c22_FC_LocalSimple_mean1_tauresrat', 'sensor_11__c22_FC_LocalSimple_mean1_tauresrat', 'sensor_11__c22_SB_BinaryStats_diff_longstretch0', ('sensor_01_sm', ''), 'sensor_12__c22_FC_LocalSimple_mean1_tauresrat', ('skew_sensor_01', ''), 'sensor_12__c22_SB_BinaryStats_diff_longstretch0', 'sensor_06__c22_SB_BinaryStats_mean_longstretch1', 'sensor_05__c22_CO_Embed2_Dist_tau_d_expfit_meandiff', 'sensor_03__permutation_entropy__dimension_4__tau_1', ('sensor_06_sm', ''), ('sensor_08_sm', ''), ('sensor_08_iqr', ''), 'sensor_11__variance', ('q50_sensor_08', ''), 'sensor_03__cid_ce__normalize_True', 
             'sensor_02__number_crossing_m__m_0', 'sensor_02__cwt_coefficients__coeff_45__w_10__widths_(2, 5, 10, 20)', ('sensor_07_sm', ''),
             'sensor_07__ar_coefficient__coeff_8__k_10', 'sensor_01__partial_autocorrelation__lag_1', 'sensor_00__c22_MD_hrv_classic_pnn40', 
             'sensor_00__c22_SB_BinaryStats_diff_longstretch0', 'sensor_05__c22_SB_BinaryStats_mean_longstretch1', 'sensor_00__c22_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1',
             'sensor_01__number_cwt_peaks__n_5', 'sensor_01__c22_MD_hrv_classic_pnn40', 'sensor_01__c22_SB_BinaryStats_diff_longstretch0', 'sensor_01__fourier_entropy__bins_2',
             'sensor_02__c22_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1', ('sensor_05_sm', ''), 'sensor_04__c22_CO_HistogramAMI_even_2_5', 'sensor_04__c22_PD_PeriodicityWang_th0_01', 
             'sensor_05__c22_CO_HistogramAMI_even_2_5', 'sensor_05__c22_MD_hrv_classic_pnn40', ('sensor_04_std_up', ''), 'sensor_03__c22_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1', 'sensor_02__autocorrelation__lag_2', 'sensor_00__agg_autocorrelation__f_agg_"mean"__maxlag_10', 'sensor_07__c22_SB_BinaryStats_diff_longstretch0', 'sensor_07__c22_SB_BinaryStats_mean_longstretch1', 'sensor_09__partial_autocorrelation__lag_1', 'sensor_01__fourier_entropy__bins_10', 'sensor_12__permutation_entropy__dimension_4__tau_1', 'sensor_08__index_mass_quantile__q_0.9', 'sensor_08__index_mass_quantile__q_0.8', 'sensor_03__c22_SB_MotifThree_quantile_hh', 'sensor_00__c22_SB_TransitionMatrix_3ac_sumdiagcov', 'sensor_12__index_mass_quantile__q_0.7', 'sensor_02__autocorrelation__lag_5', 'sensor_09__c22_MD_hrv_classic_pnn40', 'sensor_03__fourier_entropy__bins_10', 'sensor_01__c22_SB_TransitionMatrix_3ac_sumdiagcov', 'sensor_00__c22_SB_BinaryStats_mean_longstretch1', 'sensor_09__spkt_welch_density__coeff_8', 'sensor_06__mean', 'sensor_02__ar_coefficient__coeff_6__k_10', 'sensor_09__permutation_entropy__dimension_6__tau_1', ('kurtosis_sensor_05', ''), 'sensor_11__time_reversal_asymmetry_statistic__lag_3', 'sensor_12__c22_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1', 'sensor_05__autocorrelation__lag_1', 'sensor_00__c22_CO_HistogramAMI_even_2_5', ('std_sensor_01', ''), 'sensor_07__partial_autocorrelation__lag_8', 'sensor_05__cwt_coefficients__coeff_27__w_2__widths_(2, 5, 10, 20)', 'sensor_07__partial_autocorrelation__lag_9', 'sensor_09__c22_SP_Summaries_welch_rect_centroid', 'sensor_09__c22_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1', 'sensor_02__maximum', 'sensor_04__mean_abs_change', 'sensor_04__variance', 'sensor_05__cid_ce__normalize_True', 'sensor_11__index_mass_quantile__q_0.2', 'sensor_11__index_mass_quantile__q_0.1', 'sensor_11__fourier_entropy__bins_5', 'sensor_04__ar_coefficient__coeff_3__k_10', 'sensor_01__variance', ('q50_sensor_02', ''), 'sensor_11__c22_SB_TransitionMatrix_3ac_sumdiagcov', ('mean_sensor_06', ''), 'sensor_01__number_crossing_m__m_0', 'sensor_08__quantile__q_0.7', 'sensor_01__spkt_welch_density__coeff_5', 'sensor_07__c22_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1', 'sensor_07__c22_MD_hrv_classic_pnn40', 'sensor_06__variance', 'sensor_07__c22_CO_HistogramAMI_even_2_5', ('skew_sensor_05', ''), 
             'sensor_02__cwt_coefficients__coeff_57__w_20__widths_(2, 5, 10, 20)', 'sensor_01__augmented_dickey_fuller__attr_"pvalue"__autolag_"AIC"', 
             'sensor_06__fourier_entropy__bins_2', 'sensor_02__partial_autocorrelation__lag_7', ('skew_sensor_09', ''), 'sensor_05__cid_ce__normalize_False',
             ('kurtosis_sensor_11', ''), 'sensor_06__index_mass_quantile__q_0.1', ('skew_sensor_11', ''), 'sensor_07__permutation_entropy__dimension_4__tau_1', 
             'sensor_05__fourier_entropy__bins_2', ('std_sensor_10', ''),('mean_sensor_07', ''), 'sensor_02__cwt_coefficients__coeff_48__w_10__widths_(2, 5, 10, 20)',
             ('q25_sensor_05', ''), ('q95_sensor_02', ''), 'sensor_06__partial_autocorrelation__lag_2', ('mean_sensor_02', ''), 'sensor_08__c22_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1', 
             'sensor_03__permutation_entropy__dimension_5__tau_1', 'sensor_10__time_reversal_asymmetry_statistic__lag_1', 'sensor_01__spkt_welch_density__coeff_8', 
             'sensor_11__number_cwt_peaks__n_1', 'sensor_00__fourier_entropy__bins_3', 'sensor_06__autocorrelation__lag_2', 'sensor_06__index_mass_quantile__q_0.2',
            'sensor_08__c22_SB_MotifThree_quantile_hh', 'sensor_10__c22_SB_BinaryStats_diff_longstretch0', 'sensor_05__autocorrelation__lag_5', 
             'sensor_05__autocorrelation__lag_8', 'sensor_06__c22_MD_hrv_classic_pnn40', 'sensor_06__spkt_welch_density__coeff_5', 'sensor_10__ar_coefficient__coeff_7__k_10', 
             'sensor_06__partial_autocorrelation__lag_1', 'sensor_01__time_reversal_asymmetry_statistic__lag_1', 'sensor_07__c22_SP_Summaries_welch_rect_centroid', 
             'sensor_07__spkt_welch_density__coeff_8', 'sensor_03__partial_autocorrelation__lag_8', 'sensor_05__permutation_entropy__dimension_3__tau_1', 
             'sensor_10__c22_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1', 'sensor_07__index_mass_quantile__q_0.2', 'sensor_11__fourier_entropy__bins_100', 
             'sensor_01__index_mass_quantile__q_0.2','sensor_09__c22_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1', 'sensor_02__quantile__q_0.7', 'sensor_12__index_mass_quantile__q_0.9', 'sensor_05__spkt_welch_density__coeff_8', 'sensor_08__partial_autocorrelation__lag_6', 'sensor_11__fourier_entropy__bins_10', 'sensor_03__c22_CO_Embed2_Dist_tau_d_expfit_meandiff', 'sensor_03__fourier_entropy__bins_100', ('q25_sensor_08', ''), ('q50_sensor_01', ''), 'sensor_09__permutation_entropy__dimension_3__tau_1', 'sensor_01__autocorrelation__lag_2', 'sensor_04__cid_ce__normalize_False']
train_feats.drop(drop_cols,axis =1, inplace = True)
test_feats.drop(drop_cols,axis =1,inplace = True)

# Downcasting 

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
 
    return df

reduce_mem_usage(train_feats)
reduce_mem_usage(test_feats)

# Split & Scale

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_feats.drop(["sequence", "subject"], axis =1), train_lables["state"], test_size=0.33, shuffle = False)

In [ ]:
if SCALING:
    print("scaling")
    scaler= StandardScaler()
    X_train = scaler.fit_transform(X_train) 
    X_test = scaler.transform(X_test)
    test_s = scaler.transform(test_feats.drop(["sequence", "subject"], axis =1))
    
#     scaler = QuantileTransformer(n_quantiles=2000, 
#                              output_distribution='normal', 
#                              random_state=42).fit(X_train)
#     X_train = scaler.fit_transform(X_train) 
#     X_test = scaler.transform(X_test)
#     test_s = scaler.transform(test_feats.drop(["sequence", "subject"], axis =1))

# Build and Train

Model build idea from [C4RL05/V](https://www.kaggle.com/code/cv13j0/tps-apr-2022-xgboost-model#2.-Setting-the-Notebook)  

In [ ]:
params = {'n_estimators': EPOCHS,
          'max_depth': 6,
          'learning_rate': 0.015,
          'subsample': 0.95,
          'colsample_bytree': 0.60,
          'reg_lambda': 1.5,
          'reg_alpha': 6.1,
          'gamma': 6.1, #6,
          'random_state': 69,
          'objective': 'binary:logistic',
          # use hist method for leaf wise growth
          'tree_method': 'hist'  #"hist"#'gpu_hist',
         }

In [ ]:
xgb = XGBClassifier(**params,use_label_encoder=False )
xgb.fit(X_train, y_train, eval_set = [(X_test, y_test)], eval_metric = ['auc'], early_stopping_rounds = 128, verbose = 50)

In [ ]:
val_preds = xgb.predict_proba(X_test)
train_preds = xgb.predict_proba(X_train)

print("Validation AUC:" , roc_auc_score(y_test, val_preds[:, 1] ))
print("Intrinsic AUC:", roc_auc_score(y_train, train_preds[:, 1]  ))
val_preds

In [ ]:
feat_importance = pd.DataFrame(data = xgb.feature_importances_,index = train_feats.drop(["sequence", "subject"], axis =1).columns.get_level_values(0), columns = ["feature_importance"]).sort_values(by=['feature_importance'], ascending=False)
feat_importance[feat_importance['feature_importance']>0].head(20)

In [ ]:
feat_importance[feat_importance['feature_importance']>0].tail(20)

In [ ]:
print("FEATURES with 0 importance\n")
print([col for col in feat_importance[feat_importance['feature_importance']==0].index])

# Calibration plot & recalibration
A nice blog on the subject https://wttech.blog/blog/2021/a-guide-to-model-calibration/

In [ ]:
prob_true, prob_pred = calibration_curve(y_test, val_preds[:,1], n_bins=10)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
plt.plot(prob_pred,prob_true, marker='o', linewidth=1, label='xgb model probabilities')

# reference line
line = mlines.Line2D([0, 1], [0, 1], color='black')
transform = ax.transAxes
line.set_transform(transform)
ax.add_line(line)
plt.axvline(x=0.2, color = "r")
fig.suptitle('Calibration plot')
ax.set_xlabel('Predicted probability (mean)')
ax.set_ylabel('Fraction of positives (%True  in each bin)')
plt.legend()
plt.show()

#### Notes on graph:
We have a S-shaped calibration plot

This showcases that we are:
* **Underpredicting** where the fraction of True positive values is **higher** 
* **Over predicting** where the fraction of True positive values is **lower**

i.e. if we have 20% of postive values in our data (in otherwords 20% Accuracy) we should predict 20% postive values
However (from the red line graph) we are predicting around under 30% probability  --> Overpredicting

Calibration tries to fix this by "shifting" the values using calibration methods such as:
* isotonic 
* sigmoid (Platts method) 

In [ ]:
calibrator = CalibratedClassifierCV(xgb, method = "isotonic", cv='prefit')
calibrator.fit(X_test, y_test)
cal_preds = calibrator.predict_proba(X_test)

print("Validation AUC:" , roc_auc_score(y_test, val_preds[:, 1] ))
print("calibrated AUC:" , roc_auc_score(y_test, cal_preds[:, 1] ))

# Cross Validation

In [ ]:
del xgb
del X_train
del y_train
del X_test
del y_test
del val_preds
del train_preds
del calibrator

gc.collect()

In [ ]:
X = train_feats.drop(["sequence", "subject"],axis=1)
y = train_lables["state"]

In [ ]:
kfold = KFold(n_splits = FOLDS)

In [ ]:
auc_cv = []
preds = []
feature_importances = []

for fold, (train_idx, val_idx) in enumerate (kfold.split(X,y, groups =train_pivoted.index.get_level_values("sequence"))):

    print("\n","#"*10, f"Fold {fold+1}","#"*10)
    X_train, X_test = X.iloc[train_idx] , X.iloc[val_idx]
    y_train , y_test = y.iloc[train_idx], y.iloc[val_idx]
   
    if SCALING:
        print("scaling")
        scaler= StandardScaler()
        X_train = scaler.fit_transform(X_train) 
        X_test = scaler.transform(X_test)
        test_s = scaler.transform(test_feats.drop(["sequence", "subject"],axis=1))
    else:
        test_s = test_feats.drop(["sequence", "subject"],axis=1).copy()
    
    model = XGBClassifier(**params,use_label_encoder=False)
    model.fit(X_train, y_train, eval_set = [(X_test, y_test)], eval_metric = ['auc'], early_stopping_rounds = 128, verbose = 50)
    
    #fit on full data 
    X_s = scaler.transform(X)
    print(f"Fitting on full, best estimator: {params['n_estimators']}")
    params['n_estimators'] =model.get_booster().best_iteration + 100
    XGBClassifier(**params,use_label_encoder=False)
    model.fit(X_s, y, verbose = 50)
    
    #save feature importances
    feature_importances.append(model.feature_importances_)
    
    if CALIBRATION:
        calibrator = CalibratedClassifierCV(model, method = "isotonic", cv='prefit')
        calibrator.fit(X_test, y_test)
        auc = roc_auc_score(y_test, calibrator.predict_proba(X_test)[:, 1])
        print("\n Calibration AUC:" , auc)
        preds.append(calibrator.predict_proba(test_s)[:, 1])
    else:
        auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
        print("\n Validation AUC:" , auc)
        preds.append(model.predict_proba(test_s)[:, 1])

    auc_cv.append(auc)

    del X_train
    del X_test
    del y_train
    del y_test
    del model
    #del calibrator
    
    gc.collect()

print("FINAL AUC: ", np.mean(auc_cv))

In [ ]:
importances= pd.DataFrame(index = train_feats.drop(["sequence", "subject"],axis=1).columns.get_level_values(0),
             columns = [f"fold0"])
for i in range(FOLDS):
    importances[f"fold{i}"] = feature_importances[i]
importances.to_csv("feature_importances.csv")
importances

# Submission

In [ ]:
final_preds = np.sum(preds,axis =0)/FOLDS
sub["state"] = final_preds
sub.to_csv("submission.csv")
sub

In [ ]:
plt.figure(figsize = (20,8))
sns.histplot(sub["state"])
plt.show()